In [ ]:
import os

import litellm
from dotenv import load_dotenv
from google.adk.agents import Agent, ParallelAgent, SequentialAgent
from google.adk.models.google_llm import Gemini
from google.adk.models.lite_llm import LiteLlm
from google.adk.plugins.logging_plugin import (
    LoggingPlugin,
)
from google.adk.runners import InMemoryRunner
from google.genai import types
from IPython.display import Markdown, display

from agents_intensive_capstone.agents import (
    black_hat_factory,
    blue_hat_factory,
    green_hat_factory,
    white_hat_factory,
    yellow_hat_factory,
)

c:\Praveen\Personal\git\agents-intensive-capstone-2025\.venv\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.5) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


## Configure Retry Options¶

In [2]:
load_dotenv()

# 2. Access the variable
api_key = os.getenv("GOOGLE_API_KEY")
print("✅ Gemini API key setup complete.")

retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

✅ Gemini API key setup complete.


In [3]:
litellm.use_litellm_proxy = True
gpt_model = LiteLlm(model="gpt-oss-20b")

In [4]:
gemini_model = Gemini(
    model="gemini-2.5-flash-lite",
    retry_options=retry_config,
)

In [5]:
# blue_hat = blue_hat_agent.BlueHatAgent.create(
blue_hat = blue_hat_factory.BlueHatFactory.create(
    model=gpt_model
)

In [6]:
# GREEN HAT: Creativity & Alternatives
green_hat = green_hat_factory.GreenHatFactory.create(
    model=gpt_model
)

In [7]:
# YELLOW HAT: Optimism & Benefits
yellow_hat = yellow_hat_factory.YellowHatFactory.create(
    model=gemini_model,
    search_model=gemini_model
)

In [8]:
# Define the 5 "Thinking" Agents (The Team)
# WHITE HAT: Facts & Data
white_hat = white_hat_factory.WhiteHatFactory.create(
    model=gemini_model
)

In [9]:
# BLACK HAT: Caution & Risk
black_hat = black_hat_factory.BlackHatFactory.create(
    model=gpt_model
)

## Six thinking hats

In [10]:


# RED HAT: Emotions & Intuition
red_hat = Agent(
    name="RedHat",
    model=gpt_model,
    instruction="You are the Red Hat. Focus on intuition, hunches, and emotional reaction. How does this problem make users or the team feel? You do not need to justify your feelings with logic.",
)



In [11]:
# Create the Parallel Group
# This runs all 5 agents at the same time on the user prompt
thinking_team = ParallelAgent(
    name="SixHatsBrainstorm",
    sub_agents=[white_hat, red_hat, black_hat, yellow_hat, green_hat]
)

In [12]:
# Create the Final Workflow
# First run the team (Parallel), then run the manager (Sequential)
solver_workflow = SequentialAgent(
    name="SixHatsSolver",
    sub_agents=[thinking_team, blue_hat]
)

In [13]:
runner = InMemoryRunner(
    agent=solver_workflow,
    plugins=[
        LoggingPlugin()
    ], 
)
print("✅ Runner configured")

✅ Runner configured


In [14]:
response = await runner.run_debug("Should we switch our backend database from PostgreSQL to a NoSQL solution for our startup?")  # noqa: E501



 ### Created new session: debug_session_id

User > Should we switch our backend database from PostgreSQL to a NoSQL solution for our startup?
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-88209d26-3285-40ec-aae1-e6b601fdf8b0
[logging_plugin]    Session ID: debug_session_id
[logging_plugin]    User ID: debug_user_id
[logging_plugin]    App Name: InMemoryRunner
[logging_plugin]    Root Agent: SixHatsSolver
[logging_plugin]    User Content: text: 'Should we switch our backend database from PostgreSQL to a NoSQL solution for our startup?'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-88209d26-3285-40ec-aae1-e6b601fdf8b0
[logging_plugin]    Starting Agent: SixHatsSolver
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: SixHatsSolver
[logging_plugin]    Invocation ID: e-88209d26-3285-40ec-aae1-e6b601fdf8b0
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: SixHatsBrainstorm
[logging_plugin]    Invoc

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: YellowHatAgent
[logging_plugin]    Content: function_call: google_optimist
[logging_plugin]    Token Usage - Input: 310, Output: 32
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: 1d95552d-1eb8-484a-af3c-b850981524ce
[logging_plugin]    Author: YellowHatAgent
[logging_plugin]    Content: function_call: google_optimist
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['google_optimist']
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: google_optimist
[logging_plugin]    Agent: YellowHatAgent
[logging_plugin]    Function Call ID: adk-f39ff4cd-c6f3-47db-8225-df57ec26cc01
[logging_plugin]    Arguments: {'request': 'Should we switch our backend database from PostgreSQL to a NoSQL solution for our startup?'}
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-e7f5df9d-727b-40c9-9886-e0fe515a547b
[logging_plugin]    Session ID: 528208b6-620a-4b8e-b8bb

In [28]:
print("Final Blue hat response:")
display(Markdown(response[-1].content.parts[0].text))

Final Blue hat response:


---

## 1.  Problem Statement  
**Question:** Should the startup replace its PostgreSQL backend with a NoSQL database, or keep PostgreSQL (potentially coupled with a complementary NoSQL layer)?

---

## 2.  Current Context & Facts (White Hat)

| Topic | Key facts from the White‑Hat data |
|-------|-----------------------------------|
| **PostgreSQL** | Mature, ACID‑ready, JSON support, rich extensions, strong community |
| **NoSQL** | Schema‑flexible, horizontal scaling, low‑latency for certain patterns, but often weaker consistency, limited joins, higher operational learning curve |
| **Knowledge gaps** | Actual data models, query patterns, growth forecasts, team skill level, budget, latency/throughput requirements |

---

## 3.  Emotional Landscape (Red Hat)

- **Hope / Excitement** – potential for faster iteration and scalability.  
- **Anxiety / Fear** – risk of data loss, increased complexity, budget spikes.  
- **Cognitive dissonance** – balancing familiarity with PostgreSQL against the lure of NoSQL.

---

## 4.  Optimistic Perspective (Yellow Hat)

- NoSQL enables rapid schema evolution, horizontal scaling, and very fast reads/writes for unstructured data.
- PostgreSQL’s JSON/JSONB feature set now offers a “hybrid” at‑least‑once approach to handle semi‑structured data without a tooling split.
- Decision should hinge on **specific use cases, data nature, and team expertise** rather than a blanket switch.

---

## 5.  Creative Possibilities (Green Hat)

- Graph DB for relationships, time‑series DB for metrics, event‑sourcing pipelines, in‑memory caches, multi-model databases, hybrid-cloud tiering, etc.
- These ideas illustrate that *different data contexts* can be optimized with dedicated stores while still benefiting from PostgreSQL for core transactional storage.

---

## 6.  Devil’s Advocate Risks (Black Hat)

| # | Major Risk | Practical Impact |
|---|------------|-----------------|
| 1 | Eventual consistency – stale reads can break business logic. | Potential double‑charging, regulatory non‑compliance. |
| 2 | Lack of joins – leads to data duplication and maintenance headaches. | Increased storage, write overhead. |
| 3 | Limited query languages – inefficient scans and higher cloud cost. | CPU, I/O, unexpected bills. |
| 4 | Migration fragility – schema flips, referential integrity loss. | Data corruption, long downtimes. |
| 5 | Tooling/skill gap – new monitoring, backup setup. | Weeks of onboarding. |
| 6 | Cost volatility in cloud NoSQL on‑demand. | Unexpected cash‑flow spikes. |
| 7 | Vendor lock‑in. | Future multi‑cloud or on‑prem rollouts complicated. |
| 8 | Compliance gaps (audit logs, encryption). | Legal penalties. |
| 9 | Legacy ORM / code changes. | 3‑6 months of refactor. |
|10 | Hidden bugs surfacing. | Crash‑prone production. |

---

## 7.  Decision Criteria

| Criterion | Priority | Current Status |
|-----------|----------|----------------|
| **Consistency & ACID needs** | High | Current transactional flows rely on strong consistency. |
| **Schema volatility** | Medium | Some aspects (user profiles, logs) evolve often. |
| **Latency/Throughput** | High | Must meet MVP launch SLAs. |
| **Operational simplicity** | High | Small dev/ops team. |
| **Cost predictability** | Medium | Bootstrapped environment; need stable budget. |
| **Team skillset** | Medium | Comfortable with SQL; limited NoSQL experience. |
| **Regulatory compliance** | High | Potential PCI®/GDPR concerns. |
| **Future growth** | Medium | Expect scaling in the next 12–24 months. |

---

## 8.  Synthesized Recommendations (Blue Hat)

1. **Keep PostgreSQL as the primary store**  
   *Rationale:* It satisfies ACID, joins, compliance, mature tooling, and team skill set.  
   *Benefit:* No major refactor or migration risk, stable operational profile.

2. **Add a *purpose‑specific* NoSQL layer** (optional)  
   - **When to add:**  
     * Ultra‑fast caching with Redis for session/state data.  
     * Real‑time analytics / events with a time‑series store (InfluxDB / TimescaleDB).  
     * Content or metadata that is schema‑loosely evolving, but keep transactional records in PostgreSQL.  
   - **Integration Model:**  
     * Use an event‑driven pipeline (e.g., Kafka) to synchronize denormalized data to the NoSQL store.  
     * Keep clear boundaries: transactional writes go to PostgreSQL; read‑heavy or unstructured reads source from the NoSQL layer.

3. **Plan a phased migration** (if/when the workload justifies it)  
   *Start with prototypes:*  
   - Pick a single feature (e.g., user activity feed) and prototype with NoSQL.  
   - Measure latency, cost, and developer effort.

   *Rollback readiness:*  
   - Maintain automated rollback scripts and full backups.  
   - Keep the same API contract to avoid downstream changes.

4. **Address Black‑Hat risks proactively**  
   - Use **strong consistency** options (e.g., multi‑document transactions in MongoDB or `ON`/`OFF` for DynamoDB’s consistency).  
   - Implement **automatic secondary indexing** and **monitoring dashboards** for NoSQL stores.  
   - Create a **regulatory‑audit layer** on top of NoSQL for audit logs (e.g., using DynamoDB Streams + CloudTrail).

5. **Align with strategic goals**  
   - *If rapid feature iteration* is paramount, prioritize the hybrid model.  
   - *If legal or high‑value transactional integrity* dominates, stay with PostgreSQL.

---

## 9.  Next Steps (Action Plan)

| Owner | Task | Duration | Deliverable |
|-------|------|----------|-------------|
| **Product Lead** | Define the two pilot scopes (e.g., session store, activity feed). | 1 week | Scope sheets |
| **Engineering Lead** | Set up a CI/CD pipeline to deploy both PostgreSQL + NoSQL prototypes. | 2 weeks | Pipeline repo |
| **Database Ops** | Benchmark latency & cost for pilot queries. | 1–2 weeks | Benchmark report |
| **Security Lead** | Ensure audit trail compliance for pilot NoSQL layers. | 1 week | Compliance checklist |
| **Finance** | Estimate cost models for cloud tiers (PostgreSQL + NoSQL). | 1 week | Cost projection |
| **All** | Conduct a 2‑day workshop presenting pilot results and decision matrix. | 2 days | Decision matrix and recommendation |

**Decision Point:** After pilot results and cost‑benefit review, choose:
- **Option A:** Keep PostgreSQL + targeted NoSQL services (most likely).  
- **Option B:** Full migration to NoSQL (only if pilot proves cost‑effective, consistent, and achieves required performance).

---

## 10.  Summary of Key Insights

| Hat | Highlighted Point |
|-----|--------------------|
| **White** | PostgreSQL: mature, ACID, JSON support; NoSQL: flexible, horizontally scalable. |
| **Red** | Emotional tension between excitement and risk. |
| **Yellow** | Optimism for NoSQL, but realistic path through hybrid approach. |
| **Green** | Creative architectures demonstrating “what if” data worlds. |
| **Black** | Concrete failure points: consistency, migration complexity, cost volatility. |

---

**Final Decision Tendency:** Adopt a hybrid approach—retain PostgreSQL for core workload; selectively add NoSQL services for specific patterns that PostgreSQL cannot satisfy efficiently. This balances *operational safety* with *scalability agility*, aligns with the startup’s current expertise, and keeps the path to a fully NoSQL world open but deliberate.